In [1]:
import pandas as pd
import numpy as np


## FUNCIONES PARA LIMPIEZA DE DATOS 

In [34]:
def leer_dataframe(ruta):
    return pd.read_csv(ruta)
#eliminar valores nulos
def drop_null(dataframe, axis = 0):
    return dataframe.dropna(axis= axis )
## rellenar los valores que son nulos con 0
def replace(dataframe):
    return dataframe.fillna(0)


In [40]:
data = leer_dataframe("../Prueba-apoyo/cardio.csv")
data.head()


,sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,M,39,4.0,No,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,F,46,2.0,No,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,M,48,1.0,Yes,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,F,61,3.0,Yes,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,F,46,3.0,Yes,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [ ]:
#vemos que tipos de datos hay en el dataframe
print(data.info())

In [ ]:
#vemos y sumamos cuantos valores son nulos
print(data.isnull().sum())

In [ ]:
#eliminamos las filas que son nulas
data_clean= drop_null(data)
print(data_clean.isnull().sum())

### <span style="color:red">Para las variables cuantitativas vamos a escoger 3 variables cuantitativas: Edad, Nivel de colesterol total y Presión arterial sistólica</span>

#### para esto le haremos estudios de analisis descriptivo en la que calcularemos la media, mediana, desviacion estandar, rango y varianza para seguir haciendo el estudo de las varibles seleccionadas 

In [44]:
media =data_clean['age'].mean()

print (media)


49.5574398249453
